Okay, so here it is - the first Jupyter notebook. 
As a programmer in my past life and a strategy consultant for a while now, I feel strongly that is the future of consulting and most of knowledge economy. 
More on that later!
For now, what we have at hand is a hobby project.
Along with my friend, Arpan, I am trying to explore an institutional way to understand leanings of media on crucial matters.
We start with a topical one - is hydroxychloraquin (ain't that a mouthful) effective for Covid19? 

So, starting off, I want to test out how I can observe media quickly. 
As someone has said, good programmers develop algorithms, great programmers search github.
Shamelessly borrowing from the good folks at newpaper library, here is a quick render of summarizing a web article using this beautiful library. 
Hat tip folks!

In [13]:
import nltk
from newspaper import Article
article = Article('https://www.thehindu.com/sci-tech/health/coronavirus-hydroxychloroquine-not-effective-against-covid-19-says-study/article31406034.ece')
article.download()
article.parse()
article.nlp()
print(article.authors)
print(article.publish_date)
article.keywords
print(article.summary)


['As Many As Districts Have At Least One Case']
None
Hydroxychloroquine, the antimalarial drug which the U.S. President Donald Trump called a “game changer” against coronavirus (COVID-19), is not effective against the disease, according to a study.
The yet to be peer-reviewed study, published in the preprint server medRxiv, made a retrospective analysis of patients hospitalised with COVID-19 in all the Veterans Health Administration medical centers across the U.S.
According to the researchers, hydroxychloroquine, alone or in combination with azithromycin, is being widely used in COVID-19 therapy based on anecdotal evidence.
Also Read Coronavirus | Pharma units in limbo amid confusion over hydroxychloroquine exportsBased on the findings, the scientists said there is no evidence that use of hydroxychloroquine reduced the risk of mechanical ventilation in patients hospitalised with COVID-19.
“The findings from this retrospective study suggest caution in using hydroxychloroquine in hospita

In [70]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
from urllib.parse import urlparse
g_clean = [ ] 
url ='https://www.google.com/search?source=hp&ei=WijKXvTlCJHf9QPXm6D4DQ&q=is+hydroxychloroquine+working+for+covid+19&oq=is+hydroxychloroquine+working+for+&gs_lcp=CgZwc3ktYWIQAxgAMgIIADoFCAAQgwE6BwgAEEYQ_wE6DggAEOoCELQCEJoBEOUCUOYJWPQ2YIJAaAVwAHgAgAGOAYgBwxCSAQQwLjE4mAEAoAEBqgEHZ3dzLXdperABBg&sclient=psy-ab'
    
html = requests.get(url)
if html.status_code==200:
    soup = BeautifulSoup(html.text, 'lxml')
    a = soup.find_all('a') 
    for i in a:
        k = i.get('href')
        #print(k)

In [ ]:
That worked well! 
I looked at the summary code under the hood and it basically scores the sentences with the highest appreance of keywords (words appearing the most and not the usual stopwords) as high and then builds a summary taking the highest ranked sentences. Quite elegant but something to be played around with the NLP inclined.

Phew, that seemed to work but the render is not pretty at all. It is something like a lot of blob of this:
/?sa=X&ved=0ahUKEwi0zJ78gszpAhVMwzgGHYGZCC0QOwgC
/?output=search&ie=UTF-8&sa=X&ved=0ahUKEwi0zJ78gszpAhVMwzgGHYGZCC0QPAgE
/search?q=is+hydroxychloroquine+working+for+covid+19&ie=UTF-8&gbv=1&sei=cCjKXrTUAsyG4-EPgbOi6AI
# Lots of gibberish with some sembalance of links in there
//policies.google.com/privacy?fg=1
//policies.google.com/terms?fg=1    

So I revise the code to make it more meaningful

In [49]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
from urllib.parse import urlparse
g_clean = [ ] 
url ='https://www.google.com/search?source=hp&ei=WijKXvTlCJHf9QPXm6D4DQ&q=is+hydroxychloroquine+working+for+covid+19&oq=is+hydroxychloroquine+working+for+&gs_lcp=CgZwc3ktYWIQAxgAMgIIADoFCAAQgwE6BwgAEEYQ_wE6DggAEOoCELQCEJoBEOUCUOYJWPQ2YIJAaAVwAHgAgAGOAYgBwxCSAQQwLjE4mAEAoAEBqgEHZ3dzLXdperABBg&sclient=psy-ab'
    
html = requests.get(url)
if html.status_code==200:
    soup = BeautifulSoup(html.text, 'lxml')
    a = soup.find_all('a') 
    for i in a:
        k = i.get('href')
        try:
            m = re.search("(?P<url>https?://[^\s]+)", k)
            n = m.group(0)
            rul = n.split('&')[0]
            domain = urlparse(rul)
            if(re.search('google.com', domain.netloc)):
                continue
            else:
                g_clean.append(rul)
        except:
            continue
            
            
for x in range(len(g_clean)):
    print(g_clean[x])


https://www.cdc.gov/coronavirus/2019-ncov/hcp/therapeutic-options.html
https://www.goodrx.com/blog/coronavirus-medicine-chloroquine-hydroxychloroquine-as-covid19-treatment/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/%23sec0001title
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/%23sec0002title
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/%23sec0018title
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/%23sec0013title
https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30313-3/fulltext
https://www.statnews.com/pharmalot/2020/04/28/covid19-coronavirus-hydroxycholoroquine-lupus-trump/
https://www.bbc.com/news/51980731
https://theprint.in/health/hcq-breakthrough-icmr-finds-its-effective-in-preventing-coronavirus-expands-its-use/427583/
https://www.genengnews.com/news/novartis-plans-phase-iii-trial-of-hydroxychloroquine-for-covid-19/
https://www.sciencenews.org/article/coronavirus-covid-19-polit

That is good progress - makes you wonder how it has become to code! 
Although also makes you wonder how people can easily copy paste code, and pass off as the developers of that. 
Phew, and I had tried to laboriously read Knuth in 2008/09

So now, for the big one, I am going to take the top 10 results and summarize them. 
However, I do see that there are duplicates here - so first a clean pass to get non-duplicates is in order.The elegant way was to convert the list into a dictionary with keys as list elements (ignore if this didn't make sense, i just found it clever probably becuase I am getting to this after a long while)
Once I was past it, it inserted the summarizer code from newpaper api with a loop across all links - however, NCBI and goodrx blocked me.
Quite conservatively, I tried other links one-by-one (being very wary of the Google Gods aka blocking algorithms)
As next measure, I gave some cushion as exception handling to the newspaper download code. However, I had to further improvise the code to detect 'Access Denied' and not summarize those results since it was still blowing past the exceptions.
The ideal way would however, been to fork out a version of newpaper and play around with the exception handling there. 
Anyway, the code did work and I do have a large para with summary from multiple places. Code and output follow.

In [91]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
from urllib.parse import urlparse
g_clean = [ ] 
url ='https://www.google.com/search?source=hp&ei=WijKXvTlCJHf9QPXm6D4DQ&q=is+hydroxychloroquine+working+for+covid+19&oq=is+hydroxychloroquine+working+for+&gs_lcp=CgZwc3ktYWIQAxgAMgIIADoFCAAQgwE6BwgAEEYQ_wE6DggAEOoCELQCEJoBEOUCUOYJWPQ2YIJAaAVwAHgAgAGOAYgBwxCSAQQwLjE4mAEAoAEBqgEHZ3dzLXdperABBg&sclient=psy-ab'
    
html = requests.get(url)
if html.status_code==200:
    soup = BeautifulSoup(html.text, 'lxml')
    a = soup.find_all('a') 
    for i in a:
        k = i.get('href')
        try:
            m = re.search("(?P<url>https?://[^\s]+)", k)
            n = m.group(0)
            rul = n.split('&')[0]
            domain = urlparse(rul)
            if(re.search('google.com', domain.netloc)):
                continue
            else:
                g_clean.append(rul)
        except:
            continue
            
dedup_linkset = list(dict.fromkeys(g_clean))

#print(dedup_linkset)
summary = [ ]            
for x in range(len(dedup_linkset)):
  #print(dedup_linkset[x])
  try:
    article = Article(dedup_linkset[x])
    article.download()
    article.parse()
    article.nlp()
    #print(article.authors)
    #print(article.publish_date)
    article.keywords
    #print(article.summary)
    #splitted = article.summary.split()
    #first = splitted[0]
    #second = splitted[1]
    article_summary = str(article.summary)
    #print(article_summary)
    if (article_summary.find('Access Denied') < 0):
        print(article_summary)
    #if 'Access Denied' in article_summary
        #continue
    #else     
        #summary[x]= article_summary
        #print(article_summary)
  except:
    continue
    



Updated April 25, 2020There are no drugs or other therapeutics presently approved by the U.S. Food and Drug Administration (FDA) to prevent or treat COVID-19.
Current clinical management includes infection prevention and control measures and supportive care, including supplemental oxygen and mechanical ventilatory support when indicated.
The National Institutes of Health have published interim guidelines for the medical management of COVID-19external icon prepared by the COVID-19 Treatment Guidelines Panel.
These guidelines contain information about investigational therapeutics and will be updated as new information emerges and drugs and other therapeutic interventions are approved for use by FDA.
Persons seeking information about registered clinical trials for COVID-19 in the United States can search for such information here: ClinicalTrials.govexternal icon.
Studies are underway to examine if hydroxychloroquine (and a related drug chloroquine) are effective against the coronavirus.
P

With this done, what I want to do next is to figure out what is the dominant sentiment across this para for which I need to further research.

Other hat tips:
    https://simpleanalytical.com/sharing-jupyter-notebook-github
    https://towardsdatascience.com/scrape-and-summarize-news-articles-in-5-lines-of-python-code-175f0e5c7dfc
    https://newspaper.readthedocs.io/en/latest/
    https://www.w3schools.com/python/python_howto_remove_duplicates.asp
        